Use this utlity to update the returns and std_dev fields within investment-options.csv

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import brownbear as bb

# Format price data.
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [3]:
# Set size of inline plots.
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Globals

In [4]:
# Set refresh_timeseries=True to download timeseries.  Otherwise /symbol-cache is used.
refresh_timeseries = True

In [5]:
# Read in investment options input csv.
df = pd.read_csv('investment-options-in.csv', skip_blank_lines=True, comment='#')
symbols = list(df['Investment Option'])
df

,Investment Option,Description,Asset Class,1 mo,3 mo,1 Yr,3 Yr,5 Yr,Vola,DS Vola,Std Dev
0,ROAD,Construction Partners Inc,US Stocks:Industrials,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,EHC,Encompass Health,US Stocks:Healthcare,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,RF,Regions Financial,US Stocks:Financials,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,VMC,Vulcan Materials Company,US Stocks:Materials,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ADTN,ADTRAN,US Stocks:Technology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,PRA,ProAssurance Corporation,US Stocks:Financials,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,TBRG,"TruBridge, Inc.",US Stocks:Healthcare,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,MPW,"Medical Properties Trust, Inc.",US Stocks:Real Estate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,LAKE,"Lakeland Industries, Inc.",US Stocks:Consumer Staples,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Eliminate repeat symbols.
symbols = set(list(symbols))

In [7]:
# Get the timeseries for the symbols and compile into a single csv.
bb.fetch_timeseries(symbols, refresh=refresh_timeseries)
bb.compile_timeseries(symbols)

VMC ROAD PRA MPW ADTN EHC LAKE TBRG RF 


In [8]:
# Read symbols timeseries into a dataframe.
df = pd.read_csv('symbols-timeseries.csv', skip_blank_lines=True, comment='#')
df.set_index("Date", inplace=True)
df

,VMC,ROAD,PRA,MPW,ADTN,EHC,LAKE,TBRG,RF
Date,,,,,,,,,
2015-01-02,61.65,NaN,31.22,6.74,17.72,25.14,9.17,50.86,7.49
2015-01-05,59.95,NaN,30.77,6.90,17.69,25.06,8.82,51.00,7.23
2015-01-06,59.81,NaN,30.81,6.85,17.43,24.86,8.61,50.80,6.99
2015-01-07,60.94,NaN,31.39,6.84,17.51,25.63,8.75,51.10,7.03
2015-01-08,62.85,NaN,31.73,7.07,17.45,26.23,8.84,51.88,7.14
...,...,...,...,...,...,...,...,...,...
2024-12-26,264.55,91.49,15.96,3.76,8.49,94.17,24.35,19.02,23.99
2024-12-27,262.36,89.86,15.66,3.73,8.21,93.42,24.72,19.22,23.74
2024-12-30,258.16,89.38,15.54,3.72,8.30,92.25,25.50,19.32,23.66


In [9]:
annual_returns = bb.annualized_returns(df, timeperiod='daily', years=1)
annual_returns

VMC     15.11
ROAD   106.93
PRA     13.84
MPW    -12.65
ADTN    17.09
EHC     37.76
LAKE    41.26
TBRG    83.41
RF      24.83
dtype: float64

In [10]:
# Calculate 1 month, 3 months, 1 year, 3 year, and 5 year annualized returns.
annual_returns_1mo = bb.annualized_returns(df, timeperiod='daily', years=1/12)
annual_returns_3mo = bb.annualized_returns(df, timeperiod='daily', years=3/12)
annual_returns_1yr = bb.annualized_returns(df, timeperiod='daily', years=1)
annual_returns_3yr = bb.annualized_returns(df, timeperiod='daily', years=3)
annual_returns_5yr = bb.annualized_returns(df, timeperiod='daily', years=5)

In [11]:
# Calculate 20 day annualized volatility.
daily_returns = df.pct_change()
years = bb.TRADING_DAYS_PER_MONTH / bb.TRADING_DAYS_PER_YEAR
vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years)

In [12]:
# Calculate 20 day annualized downside volatility.
ds_vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years, downside=True)

In [13]:
# Resample df on a monthly basis.
df.index = pd.to_datetime(df.index)
monthly = df.resample('ME').ffill()

In [14]:
# Calculate monthly returns.
monthly_returns = monthly.pct_change()

In [15]:
# Calculate 3 year annualized standard deviation.
std_dev = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=3)

In [16]:
# Read investment-options-in.csv
lines = []
with open('investment-options-in.csv', 'r') as f:
    lines = [line.strip() for line in f]
#lines

In [17]:
# For each etf, write out the 1 Yr, 3 Yr, 5 Yr, and std dev.
out = []
for line in lines:
    # Copy empty and comment lines.
    if not line or line.startswith('#'):
        out.append(line)
        continue
    # Split line by comma delimiter; strip the fields.
    fields = line.split(',')
    fields = [field.strip() for field in fields]
    symbol = fields[0].strip('\"')
    # Copy header.
    if symbol == 'Investment Option':
        out.append(line)
        continue
    ret_1mo = annual_returns_1mo[symbol]
    ret_3mo = annual_returns_3mo[symbol]
    ret_1yr = annual_returns_1yr[symbol]
    ret_3yr = annual_returns_3yr[symbol]
    ret_5yr = annual_returns_5yr[symbol]
    
    if np.isnan(ret_3yr): ret_3yr = ret_1yr
    if np.isnan(ret_5yr): ret_5yr = ret_3yr

    _vola = vola[symbol]*100
    _ds_vola = ds_vola[symbol]*100
    sd = std_dev[symbol]*100

    out.append('{},{},{},"{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}"'
               .format(fields[0], fields[1], fields[2],
                       ret_1mo, ret_3mo, ret_1yr, ret_3yr, ret_5yr, _vola, _ds_vola, sd)) 

In [18]:
# Write out asset-classes.csv
with open('investment-options.csv', 'w') as f:
    for line in out:
        f.write(line + '\n')